In [1]:
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset

In [2]:
tokenizer = CLIPTokenizer.from_pretrained("stabilityai/sdxl-turbo", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("stabilityai/sdxl-turbo", subfolder="text_encoder")#.to('cuda')

In [ ]:
laion = load_dataset("laion/laion400m_new")

In [ ]:
rnd = laion['train'].shuffle(seed=5)

In [ ]:
subset = rnd.select(range(5000))

In [ ]:
test = subset.select(range(500))

In [ ]:
captions = subset["caption"]

In [ ]:
captions[1]

In [ ]:
len(captions)

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
import torch
tokenizer = CLIPTokenizer.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="text_encoder", use_safetensors=True)
text_encoder.to('cuda')
""

In [ ]:
text_input = tokenizer(captions, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors='pt')

In [ ]:
with torch.no_grad():
    layers = text_encoder(text_input.input_ids.to("cuda"))

In [ ]:
text_embeddings = layers.pooler_output

In [ ]:
text_embeddings.shape

In [ ]:
import umap
reducer = umap.UMAP()

In [ ]:
text_emb_cpu = text_embeddings.cpu()

In [ ]:
text_emb_cpu.shape

In [ ]:
mapping = reducer.fit(text_emb_cpu)

In [ ]:
mapping.transform(text_emb_cpu[0:1])

In [ ]:
from utils.data import descriptions
prompts = descriptions[:,0]
prompts

In [ ]:
from utils.model import CLIP
clip = CLIP()
embeddings = umap.embed(prompts)

In [ ]:
import joblib
path = 'dump/umap.sav'
joblib.dump(mapping, path)